In [36]:
from dataclasses import dataclass
import typing as t 

import gymnasium as gym
import energydatamodel as edm
import enerflow as ef

import numpy as np
import pandas as pd
import xarray as xr
import statsmodels.formula.api as smf
from statsmodels.iolib.smpickle import load_pickle

import folium
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

In [86]:
m = folium.Map(location=[53.5, 0], zoom_start=6.4)
folium.GeoJson("data/pes10.geojson", name='geopolygon').add_to(m)

def add_markers(lats, lons):
for lat in dwd_Hornsea1.latitude.values:
    for lon in dwd_Hornsea1.longitude.values:
        folium.CircleMarker(location=[lat, lon], radius=0.1, color='red').add_to(m)

m

# Preprocessing data

In [67]:
dwd_Hornsea1.longitude

<xarray.DataArray 'longitude' (longitude: 6)> Size: 48B
array([1.702, 1.767, 1.832, 1.897, 1.962, 2.027])
Coordinates:
  * longitude  (longitude) float64 48B 1.702 1.767 1.832 1.897 1.962 2.027

In [65]:
#dwd_Hornsea1.coords['latitude']
dwd_Hornsea1.coords['longitude']

<xarray.DataArray 'longitude' (longitude: 6)> Size: 48B
array([1.702, 1.767, 1.832, 1.897, 1.962, 2.027])
Coordinates:
  * longitude  (longitude) float64 48B 1.702 1.767 1.832 1.897 1.962 2.027

In [53]:
dwd_Hornsea1 = xr.open_dataset("data/dwd_icon_eu_hornsea_1_20200920_20231027.nc")
dwd_Hornsea1_features = dwd_Hornsea1["WindSpeed:100"].mean(dim=["latitude","longitude"]).to_dataframe().reset_index()
dwd_Hornsea1_features["ref_datetime"] = dwd_Hornsea1_features["ref_datetime"].dt.tz_localize("UTC")
dwd_Hornsea1_features["valid_datetime"] = dwd_Hornsea1_features["ref_datetime"] + pd.TimedeltaIndex(dwd_Hornsea1_features["valid_datetime"],unit="hours")

/var/folders/sj/xlrs69b568n4mntwz_7crr4r0000gn/T/ipykernel_51666/835483287.py:4: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  dwd_Hornsea1_features["valid_datetime"] = dwd_Hornsea1_features["ref_datetime"] + pd.TimedeltaIndex(dwd_Hornsea1_features["valid_datetime"],unit="hours")


In [3]:
dwd_solar = xr.open_dataset("data/dwd_icon_eu_pes10_20200920_20231027.nc")
dwd_solar_features = dwd_solar["SolarDownwardRadiation"].mean(dim="point").to_dataframe().reset_index()
dwd_solar_features["ref_datetime"] = dwd_solar_features["ref_datetime"].dt.tz_localize("UTC")
dwd_solar_features["valid_datetime"] = dwd_solar_features["ref_datetime"] + pd.TimedeltaIndex(dwd_solar_features["valid_datetime"],unit="hours")

/var/folders/sj/xlrs69b568n4mntwz_7crr4r0000gn/T/ipykernel_51666/167480448.py:4: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  dwd_solar_features["valid_datetime"] = dwd_solar_features["ref_datetime"] + pd.TimedeltaIndex(dwd_solar_features["valid_datetime"],unit="hours")


In [4]:
energy_data = pd.read_csv("data/Energy_Data_20200920_20231027.csv")
energy_data["dtm"] = pd.to_datetime(energy_data["dtm"])
energy_data["Wind_MWh_credit"] = 0.5*energy_data["Wind_MW"] - energy_data["boa_MWh"]
energy_data["Solar_MWh_credit"] = 0.5*energy_data["Solar_MW"]

In [5]:
data = dwd_Hornsea1_features.merge(dwd_solar_features,how="outer",on=["ref_datetime","valid_datetime"])
data = data.set_index("valid_datetime").groupby("ref_datetime").resample("30T").interpolate("linear")
data = data.drop(columns="ref_datetime",axis=1).reset_index()
data = data.merge(energy_data,how="inner",left_on="valid_datetime",right_on="dtm")
data = data[data["valid_datetime"] - data["ref_datetime"] < np.timedelta64(50,"h")]
data.rename(columns={"WindSpeed:100":"WindSpeed"},inplace=True)
data["total_generation_MWh"] = data["Wind_MWh_credit"] + data["Solar_MWh_credit"]


/var/folders/sj/xlrs69b568n4mntwz_7crr4r0000gn/T/ipykernel_51666/3095911322.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  data = data.set_index("valid_datetime").groupby("ref_datetime").resample("30T").interpolate("linear")


# Create Environment

In [6]:
class HEFTCom2024Environment(gym.Env):

    def __init__(self, 
                 data: pd.DataFrame, 
                 train_end: t.Optional[pd.Timestamp] = None):
        
        self.data = data
        self.train_end = train_end if train_end is not None else data["valid_datetime"].max()

    def reset(self):
        
        initial_data = self.data[self.data["valid_datetime"] <= self.train_end]
        
        return initial_data

    def step(): 
        pass


In [7]:
from enerflow.problems.objectives import PinballLoss
from enerflow.problems.objectives import BaseScore

class MarketRevenue(BaseScore):

    def calculate(self, bid, production, day_ahead_price, single_system_price):
        revenue = bid * day_ahead_price + (production - bid) * (single_system_price - 0.07 * (production - bid))

        return revenue

In [8]:
class Predictor(ef.Predictor):
        def __init__(self):
            self.model_name = "quantreg"
            self.quantiles = range(10,100,10)
            self.models = dict()

        def save_model(self, path: str):
            for quantile in self.quantiles:
                self.models[f"q{quantile}"].save(f"{path}/{self.model_name}_q{quantile}.pickle")
        
        def load_model(self, path: str):
            for quantile in self.quantiles:
                self.models[f"q{quantile}"] = load_pickle(f"{path}/{self.model_name}_q{quantile}.pickle")

        def train(self, data: pd.DataFrame):
            data = data[data["SolarDownwardRadiation"].notnull()]
            data = data[data["WindSpeed"].notnull()]
            
            mod = smf.quantreg('total_generation_MWh ~ bs(SolarDownwardRadiation,df=5) + bs(WindSpeed,df=8)',
                               data=data)
    
            for quantile in range(10,100,10):
                print(quantile)
                self.models[f"q{quantile}"] = mod.fit(q=quantile/100, max_iter=2500)

        def predict(self, input: pd.DataFrame) -> pd.DataFrame:
            prediction = pd.DataFrame()

            for quantile in range(10,100,10):
                prediction[f"q{quantile}"] = self.models[f"q{quantile}"].predict(input)
                prediction.loc[prediction[f"q{quantile}"] < 0, f"q{quantile}"] = 0
                
            return prediction
    

In [18]:
class Optimizer(ef.Optimizer):
    
    def optimize(self, prediction): 
        bid = prediction["q50"]
        
        return bid

In [19]:
class Agent(ef.Agent):
    
    def __init__(self):
        self.predictor = Predictor()
        self.optimizer = Optimizer()

    def load_predictor(self, path: str):
        self.predictor.load_model(path)

    def train(self, data: pd.DataFrame):
        self.predictor.train(data)

    def act(self, data: pd.DataFrame):
        prediction = self.predictor.predict(data)
        bid = self.optimizer.optimize(prediction)
        return bid

In [20]:
env = HEFTCom2024Environment(data)
initial_data = env.reset()

agent = Agent()
agent.load_predictor("./models")
bid = agent.act(initial_data)

In [29]:
y_true = initial_data[["total_generation_MWh"]].values
y_preds = agent.predictor.predict(initial_data).values

pinball = PinballLoss(quantiles=np.arange(0.1,1,0.1))
scores = pinball.score(y_true, y_preds)
scores

29.51707725012343

In [35]:
scorer = MarketRevenue()
revenue = scorer.calculate(bid=bid.values, 
                           production=initial_data["total_generation_MWh"].values, 
                           day_ahead_price=initial_data["DA_Price"].values, 
                           single_system_price=initial_data["SS_Price"].values)
np.nansum(revenue)

21335870482.101124

In [66]:
# Next step
# 3) Create a validation dataset and run the agent 
# 4) (Optional) Create the energy system model (and create a plot of the energy system model)

# 1) Clean up dataset so that it is only the relevant data (make a create_dataset.py file)
# 2) Run the agent in "backtest" mode and plot the generation, forecast, bids and revenue for every market increment
# 3) Now run the trained agent in a loop
# 4) Glue together the backtest and the backtest and make a bar plot of the revenue (backtest and loop)